# QF605 | Fixed Income Securities
## Project

#### Members
1. Jung Hyung-yun
2. Lim Jeng
3. Nguyen Ngo Duy Quang
4. Shao Jiayu
5. Sun Qiaozhen
6. Tan Hui Shan
7. Tan Wei Hao

In [6]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d,RegularGridInterpolator
from scipy.integrate import quad
from scipy.stats import norm

In [7]:
dfs=pd.read_excel('Data_1_Output.xlsx',sheet_name=None)

In [8]:
dfs.keys()

dict_keys(['oisDiscFactors', 'liborDiscFactors', 'fwdSwapRates'])

In [9]:
dfs['fwdSwapRates']

,Start,Tenor,liborForwardSwapRate
0,1Y,1Y,0.032007
1,1Y,2Y,0.033259
2,1Y,3Y,0.034011
3,1Y,5Y,0.035255
4,1Y,10Y,0.038428
5,5Y,1Y,0.039274
6,5Y,2Y,0.040075
7,5Y,3Y,0.040072
8,5Y,5Y,0.041093
9,5Y,10Y,0.043634


In [10]:
df1=pd.read_excel('SABR_PART2.xlsx',sheet_name=None)

In [11]:
df1['alpha']

,1,2,3,5,10
0,0.139075,0.184650,0.196845,0.178065,0.171171
1,0.145953,0.188368,0.200221,0.181320,0.172695
2,0.152831,0.192086,0.203597,0.184575,0.174219
3,0.159709,0.195803,0.206972,0.187830,0.175743
4,0.166587,0.199521,0.210348,0.191085,0.177267
5,0.168857,0.198741,0.209943,0.193232,0.178076
6,0.171127,0.197961,0.209538,0.195379,0.178885
7,0.173397,0.197181,0.209133,0.197526,0.179694
8,0.175666,0.196402,0.208728,0.199673,0.180503
9,0.177936,0.195622,0.208323,0.201819,0.181312


In [12]:
list(df1['alpha'].index+1)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [13]:
np.array(df1['alpha'])

array([[0.13907517, 0.18465032, 0.1968453 , 0.17806526, 0.17117123],
       [0.14595313, 0.18836804, 0.20022093, 0.18132021, 0.17269524],
       [0.15283109, 0.19208576, 0.20359655, 0.18457517, 0.17421924],
       [0.15970905, 0.19580348, 0.20697218, 0.18783012, 0.17574325],
       [0.16658701, 0.1995212 , 0.21034781, 0.19108508, 0.17726725],
       [0.16885685, 0.19874129, 0.20994283, 0.19323195, 0.17807616],
       [0.17112669, 0.19796137, 0.20953784, 0.19537883, 0.17888506],
       [0.17339654, 0.19718145, 0.20913286, 0.19752571, 0.17969396],
       [0.17566638, 0.19640154, 0.20872788, 0.19967258, 0.18050286],
       [0.17793623, 0.19562162, 0.20832289, 0.20181946, 0.18131177]])

In [14]:
list[dfs['oisDiscFactors']['DiscountFactor']]

list[0     1.000000
1     0.998752
2     0.997009
3     0.995270
4     0.993531
        ...   
56    0.857703
57    0.855126
58    0.852548
59    0.849986
60    0.847424
Name: DiscountFactor, Length: 61, dtype: float64]

In [15]:
OISD = interp1d(dfs['oisDiscFactors']['TN'].values, dfs['oisDiscFactors']['DiscountFactor'].values, kind='linear')

In [16]:

LIBORD = interp1d(dfs['liborDiscFactors']['TN'].values, dfs['liborDiscFactors']['DiscountFactor'].values, kind='linear')
Alpha = RegularGridInterpolator((list(df1['alpha'].index+1), list(df1['alpha'].columns)),np.array(df1['alpha']) , method='linear',bounds_error=False,fill_value=None)
Rho = RegularGridInterpolator((list(df1['rho'].index+1), list(df1['rho'].columns)),np.array(df1['rho']) , method='linear',bounds_error=False,fill_value=None)
Nu = RegularGridInterpolator((list(df1['nu'].index+1), list(df1['nu'].columns)),np.array(df1['nu']), method='linear',bounds_error=False,fill_value=None)
def forwardswaprate(expiry, tenor, m):
    floated, fix = 0, 0
    n=int(tenor*m)
    for i in range(1,n+1):
        t=expiry+i/m
        d2=LIBORD(t)
        floated+=OISD(t)*m * ((LIBORD(t-1/m) - d2) / d2)/m
        fix+=OISD(t)/m
    return floated/fix
def sabr(F, K, T, alpha, beta, rho, nu):
    X = K
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom
    return sabrsigma
def IRR_0(K, m, tenor):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(tenor*m) )
    return value

def IRR_1(K, m, tenor):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, tenor) + 1/(K*m)*tenor*m/(1+K/m)**(tenor*m+1)
    return firstDerivative

def IRR_2(K, m, tenor):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, tenor) - 1/(K*m*m)*(tenor*m)*(tenor*m+1)/(1+K/m)**(tenor*m+2)
    return secondDerivative


def hpp(K, m, tenor):
    a=(-IRR_2(K, m, tenor)*K-2*IRR_1(K, m, tenor))/(IRR_0(K, m, tenor)**2)
    b=2*K*(IRR_1(K, m, tenor)**2)/(IRR_0(K, m, tenor)**3)
    return a+b
def IRRswaptionpay(S,K,vol,T,m):
    d1=((np.log(S/K)+(vol**2)*T/2)/(vol*np.sqrt(T)))
    d2=d1-vol*np.sqrt(T)
    Black=S*norm.cdf(d1) - K*norm.cdf(d2)
    return IRR_0(S, m, T)*Black
def IRRswaptionrec(S,K,vol,T,m):
    d1=((np.log(S/K)+(vol**2)*T/2)/(vol*np.sqrt(T)))
    d2=d1-vol*np.sqrt(T)
    Black=K*norm.cdf(-d2) - S*norm.cdf(-d1)
    return IRR_0(S, m, T)*Black
def CMS_rate(m,T,expiry):
    S=forwardswaprate(expiry, T, m)
    alpha=Alpha([[expiry,T]])
    rho=Rho([[expiry,T]])
    nu=Nu([[expiry,T]])
    f1 = lambda x:IRRswaptionpay(S,x,sabr(S, x, expiry, alpha, 0.9, rho, nu),T,m)*hpp(x, m, T)
    f2 = lambda x:IRRswaptionrec(S,x,sabr(S, x, expiry, alpha, 0.9, rho, nu),T,m)*hpp(x, m, T)
    integral= S+quad(f2,0,S)+quad(f1,S,0.1)
    return integral 
def PV_Leg(expiry, T, m):
    total_period=int(m*expiry)
    value=0
    for i in range(1,total_period+1):
        value+=OISD(i/m)*CMS_rate(m,T,i/m)
    return value

In [17]:
PV_Leg(5,10,2)[0]

0.4717847952432889

In [18]:
PV_Leg(10, 2, 4)[0]

1.523655385667613

In [19]:
oneyear=[[1,1],[1,2],[1,3],[1,5],[1,10]]
fiveyear=[[5,1],[5,2],[5,3],[5,5],[5,10]]
tenyear=[[10,1],[10,2],[10,3],[10,5],[10,10]]
expiry_tenor_pairs = oneyear + fiveyear + tenyear

In [20]:
for expiry, tenor in oneyear:
    print(f"{expiry}y × {tenor}y =", CMS_rate(2,tenor,expiry)[0])

1y × 1y = 0.03211703180202619
1y × 2y = 0.033847806713096014
1y × 3y = 0.035225802934942534
1y × 5y = 0.037552098268465624
1y × 10y = 0.04589285734032557


In [21]:
for expiry, tenor in fiveyear:
    print(f"{expiry}y × {tenor}y =", CMS_rate(2,tenor,expiry)[0])

5y × 1y = 0.03940960613875025
5y × 2y = 0.0405809374929745
5y × 3y = 0.04106967135062197
5y × 5y = 0.04305847567304777
5y × 10y = 0.05003438167755705


In [22]:
for expiry, tenor in tenyear:
    print(f"{expiry}y × {tenor}y =", CMS_rate(2,tenor,expiry)[0])

10y × 1y = 0.04232867442709992
10y × 2y = 0.04362965743028935
10y × 3y = 0.04517772816317752
10y × 5y = 0.04858124965927622
10y × 10y = 0.06163963472294749


In [23]:
cms_data = []

for expiry, tenor in expiry_tenor_pairs:
    cms_val = CMS_rate(2, tenor, expiry)[0]
    cms_data.append({"Start": expiry, "Tenor": tenor, "CMS Rate": cms_val})

cms_df = pd.DataFrame(cms_data)

In [24]:
cms_df

,Start,Tenor,CMS Rate
0,1,1,0.032117
1,1,2,0.033848
2,1,3,0.035226
3,1,5,0.037552
4,1,10,0.045893
5,5,1,0.039410
6,5,2,0.040581
7,5,3,0.041070
8,5,5,0.043058
9,5,10,0.050034


In [25]:
dfs['fwdSwapRates']

,Start,Tenor,liborForwardSwapRate
0,1Y,1Y,0.032007
1,1Y,2Y,0.033259
2,1Y,3Y,0.034011
3,1Y,5Y,0.035255
4,1Y,10Y,0.038428
5,5Y,1Y,0.039274
6,5Y,2Y,0.040075
7,5Y,3Y,0.040072
8,5Y,5Y,0.041093
9,5Y,10Y,0.043634


In [26]:
fwd_df = dfs['fwdSwapRates']
fwd_df["Start"] = fwd_df["Start"].astype(str)
fwd_df["Tenor"] = fwd_df["Tenor"].astype(str)

cms_df["Start"] = cms_df["Start"].astype(str)
cms_df["Tenor"] = cms_df["Tenor"].astype(str)

cms_df["Start"] = cms_df["Start"].astype(int).astype(str) + "Y"
cms_df["Tenor"] = cms_df["Tenor"].astype(int).astype(str) + "Y"

In [27]:
combined_df = pd.merge(fwd_df, cms_df, how="inner", on=["Start", "Tenor"])
combined_df["CMS-Fwd"] = combined_df["CMS Rate"] - combined_df["liborForwardSwapRate"]

combined_df.set_index(["Start", "Tenor"], inplace=True)
combined_df.sort_index(inplace=True)

combined_df.rename(columns={"liborForwardSwapRate": "Forward Rate"}, inplace=True)
print(combined_df)

             Forward Rate  CMS Rate   CMS-Fwd
Start Tenor                                  
10Y   10Y        0.053458  0.061640  0.008182
      1Y         0.042190  0.042329  0.000139
      2Y         0.043116  0.043630  0.000514
      3Y         0.044097  0.045178  0.001080
      5Y         0.046249  0.048581  0.002332
1Y    10Y        0.038428  0.045893  0.007465
      1Y         0.032007  0.032117  0.000110
      2Y         0.033259  0.033848  0.000589
      3Y         0.034011  0.035226  0.001215
      5Y         0.035255  0.037552  0.002297
5Y    10Y        0.043634  0.050034  0.006401
      1Y         0.039274  0.039410  0.000136
      2Y         0.040075  0.040581  0.000506
      3Y         0.040072  0.041070  0.000997
      5Y         0.041093  0.043058  0.001965


In [28]:
# Reset and clean index
comparison_df = combined_df.reset_index()

# Reorder columns
comparison_df = comparison_df[["Start", "Tenor", "Forward Rate", "CMS Rate", "CMS-Fwd"]]

# Round for display
comparison_df[["Forward Rate", "CMS Rate", "CMS-Fwd"]] = comparison_df[["Forward Rate", "CMS Rate", "CMS-Fwd"]].round(6)

# Sort: Start by years first, not alphabetically
comparison_df["Start_num"] = comparison_df["Start"].str.replace("Y", "").astype(int)
comparison_df["Tenor_num"] = comparison_df["Tenor"].str.replace("Y", "").astype(int)

comparison_df.sort_values(by=["Start_num", "Tenor_num"], inplace=True)

# Drop helper columns
comparison_df.drop(columns=["Start_num", "Tenor_num"], inplace=True)

# Reset index to remove row numbers
comparison_df.reset_index(drop=True, inplace=True)

# Final display
comparison_df

,Start,Tenor,Forward Rate,CMS Rate,CMS-Fwd
0,1Y,1Y,0.032007,0.032117,0.000110
1,1Y,2Y,0.033259,0.033848,0.000589
2,1Y,3Y,0.034011,0.035226,0.001215
3,1Y,5Y,0.035255,0.037552,0.002297
4,1Y,10Y,0.038428,0.045893,0.007465
5,5Y,1Y,0.039274,0.039410,0.000136
6,5Y,2Y,0.040075,0.040581,0.000506
7,5Y,3Y,0.040072,0.041070,0.000997
8,5Y,5Y,0.041093,0.043058,0.001965
9,5Y,10Y,0.043634,0.050034,0.006401
